In [64]:
# Combine two dataframes into one common dataframe,
# and dump to SQL file for upload to Web app
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For munging
import re
import json

In [65]:
yalePath = os.path.join(PROJ_ROOT, "data", "interim", "yale_bios_processed.csv")
harvardPath = os.path.join(PROJ_ROOT, "data", "interim", "harvard_bios_processed.csv")

yale = pd.read_csv(yalePath)
harvard = pd.read_csv(harvardPath)

In [66]:
yale.shape

(3004, 19)

In [67]:
harvard.shape

(4270, 19)

In [68]:
yale.columns

Index([u'Name', u'B/T', u'City', u'Cl.', u'Events', u'High School',
       u'Hometown/High School', u'Hometown/Region', u'Ht.', u'Name_link',
       u'No.', u'Pos.', u'Region', u'Weapon', u'Wt.', u'season', u'sport',
       u'Bio', u'headers'],
      dtype='object')

In [69]:
harvard.columns

Index([u'Name', u'B/T', u'City', u'Event', u'High School', u'Hometown', u'Ht.',
       u'Name_link', u'No.', u'Position', u'Region', u'S/C', u'Weapon', u'Wt.',
       u'Yr.', u'season', u'sport', u'headers', u'Bio'],
      dtype='object')

In [70]:
yale.head(20)

,Name,B/T,City,Cl.,Events,High School,Hometown/High School,Hometown/Region,Ht.,Name_link,No.,Pos.,Region,Weapon,Wt.,season,sport,Bio,headers
0,A.J. Haase,NaN,Bonne Terre,Fr.,NaN,North County,"Bonne Terre, MO / North County","Bonne Terre, MO",6-4,/sports/m-footbl/2006-07/bios/haase_a.j.00.html,NaN,TE,MO,NaN,245.0,2006-10,Football,2008: Started eight games at TE... LedYale end...,"{u'Position:': u'G', u'name': u'Josh Greenberg..."
1,AJ Edwards,NaN,Seattle,Fr.,NaN,South Kent School,"Seattle, Wash. / South Kent School","Seattle, Wash.",6-5,/sports/m-baskbl/2013-14/bios/edwards_aj_geat,25.0,G,Wash.,NaN,190.0,2013-17,Basketball,2013-14: Appeared in five games… Scored five p...,"{u'Position:': u'Pole Vault', u'name': u'Brend..."
2,Aarica West,NaN,Buena Park,Freshman,NaN,Vista Murrieta,"Buena Park, Calif. / Vista Murrieta","Buena Park, Calif.",5-10,/sports/w-baskbl/2009-10/bios/west_aarica,31.0,G/F,Calif.,NaN,NaN,2009-12,Basketball,Before Yale: Played four years scholasticallyi...,"{u'Hometown:': u'Concord, MA', u'High School:'..."
3,Aaron Ault,NaN,Altamonte Springs,Fr.,NaN,Lake Brantley,"Altamonte Springs, FL / Lake Brantley","Altamonte Springs, FL",6-3,/sports/m-footbl/2012-13/bios/ault_aaron_lcur,93.0,LB,FL,NaN,194.0,2012-16,Football,"2013: Played in all 10 games, mostly onspecial...","{u'Position:': u'DB', u'Height:': u'5-11', u'Y..."
4,Aaron Bosgang,NaN,Port Washington,Fr.,NaN,Paul D. Schreiber High School,"Port Washington, N.Y. / Paul D. Schreiber High...","Port Washington, N.Y.",NaN,/sports/m-crewlt/2016-17/bios/bosgang_aaron_uh2u,NaN,NaN,N.Y.,NaN,NaN,2016-17,Crew (Lightweight),Off the Water: President of National Honor Soc...,"{u'Position:': u'OL', u'Height:': u'6-6', u'Ye..."
5,Aaron Fuchs,NaN,Somerset West,Senior,NaN,Panel Vallei,"Somerset West, South Africa / Panel Vallei","Somerset West, South Africa",NaN,/sports/m-squash/2009-10/bios/fuchs_aaron,NaN,NaN,South Africa,NaN,NaN,2009-10,Squash,"Ibegan my squash adventure at the age of 14, m...","{u'name': u'Duncan Logie', u'Height:': u'6-6',..."
6,Aaron Greenberg,NaN,Eden Prairie,Fr.,Freestyle/Breaststroke/Backstroke,Eden Prairie High School,"Eden Prairie, Minn. / Eden Prairie High School","Eden Prairie, Minn.",NaN,/sports/m-swim/2013-14/bios/greenberg_aaron_ifc2,NaN,NaN,Minn.,NaN,NaN,2013-17,Swimming & Diving,Best Times: 50 Freestyle: 19.74 100 ...,"{u'Hometown:': u'Woodbridge, Conn.', u'Positio..."
7,Aaron Jones,NaN,"St. Catharines, Ont., Canada",Fr.,NaN,Governor Simcoe,"St. Catharines, Ont., Canada / Governor Simcoe","St. Catharines, Ont., Canada",NaN,/sports/m-crewlt/2013-14/bios/jones_aaron_g4f2,NaN,NaN,NOREGION?,NaN,NaN,2013-17,Crew (Lightweight),Before Yale: Rowed for St. Catharines Rowing C...,"{u'Position:': u'LB', u'Height:': u'6-3', u'Ye..."
8,Aaron Rodriguez,NaN,Cypress,Jr.,NaN,Cy-Fair,"Cypress, TX / Cy-Fair","Cypress, TX",5-5,/sports/m-soccer/2003-04/bios/rodriguez_aaron0...,7.0,Midfield,TX,NaN,135.0,2003-05,Soccer,"Rodriguez, who had an excellent spring season,...","{u'name': u""Hugh O'Cinneide"", u'Year:': u'Seni..."
9,Aaron Seriff-Cullick,NaN,-,Fr.,Diving,-,- / -,-,NaN,/sports/m-swim/2009-10/bios/seriff-cullick_aar...,NaN,NaN,-,NaN,NaN,2009-13,Swimming & Diving,2012-2013: Earned Winter Academic All-Ivy reco...,"{u'Hometown:': u'Richboro, Pa.', u'Position:':..."


In [71]:
harvard.head()

,Name,B/T,City,Event,High School,Hometown,Ht.,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport,headers,Bio
0,A.J. Jaffe,NaN,St. Charles,NaN,Marmion Academy,"St. Charles, Ill.",NaN,/sports/wrest/2016-17/bios/jaffe_aj,NaN,141,Ill.,NaN,NaN,NaN,Freshman,2016-17,wrest,"{u'Hometown:': u'St. Charles, Ill.', u'Positio...",Before Harvard Four-time letterwinner at Marmi...
1,A.J. Kennedy,NaN,Fonthill,NaN,Notre Dame College,"Fonthill, Ont.",6-3,/sports/fball/2003-04/bios/a.j._kennedy,98,DE,Ont.,NaN,NaN,220,Freshman,2003-04,fball,"{u'Position:': u'DE', u'Year:': u'Freshman', u...",NaN
2,AJ Carvalho,NaN,Swansea,NaN,Joseph Case,"Swansea, Mass.",NaN,/sports/mcrew-lw/2010-11/bios/varsity/Carvalho...,NaN,NaN,Mass.,NaN,NaN,NaN,Sophomore,2010-13,mcrew-lw,"{u'name': u'Antonio Carvalho', u'Year:': u'Sop...",Sophomore • 2011Did not see spring race action...
3,Aaki Vora,NaN,Mumbai,Freestyle,The Cathedral and John Connon School,"Mumbai, India",NaN,/sports/wswimdive/2016-17/bios/Vora_Aaki,NaN,NaN,India,NaN,NaN,NaN,Freshman,2016-17,wswimdive,"{u'Hometown:': u'Mumbai, India', u'Position:':...",Before Harvard Best female athlete at the 2015...
4,Aaron Byrd,NaN,Abilene,NaN,Cooper,"Abilene, Texas",6-2,/sports/fball/2001-02/bios/aaron_byrd,3,DB,Texas,NaN,NaN,190,Freshman,2001-03,fball,"{u'Position:': u'DB', u'Year:': u'Freshman', u...",NaN


Definitive list of columns:
1. Name
2. B/T
3. City
4. Events
5. High School
6. Hometown
7. Height
8. No.
9. Position
10. Region
11. S/C
12. Weapon
13. Weight
14. Year
15. Active Seasons
16. Sport
17. Headers
18. Bio

Next step involves adding blank data to tables when either does not contain the above columns, and cleaning up some of the data to ensure consistency.

In [72]:
yale.columns

Index([u'Name', u'B/T', u'City', u'Cl.', u'Events', u'High School',
       u'Hometown/High School', u'Hometown/Region', u'Ht.', u'Name_link',
       u'No.', u'Pos.', u'Region', u'Weapon', u'Wt.', u'season', u'sport',
       u'Bio', u'headers'],
      dtype='object')

In [73]:
# Insert missing columns
yale["S/C"] = None
yale["College"] = "Yale"
# Re-order, then rename
# Yale
yaleColumns = ["Name", "B/T", "City", "Events", "High School", "Hometown/High School",
               "Ht.", "No.", "Pos.", "Region", "S/C", "Weapon", "Wt.", "Cl.", "season", "sport", "headers", "Bio", "College"]

In [74]:
yale = yale[yaleColumns]

In [75]:
harvard.columns

Index([u'Name', u'B/T', u'City', u'Event', u'High School', u'Hometown', u'Ht.',
       u'Name_link', u'No.', u'Position', u'Region', u'S/C', u'Weapon', u'Wt.',
       u'Yr.', u'season', u'sport', u'headers', u'Bio'],
      dtype='object')

In [76]:
harvard["College"] = "Harvard"
harvardColumns = ["Name", "B/T", "City", "Event", "High School", "Hometown",
               "Ht.", "No.", "Position", "Region", "S/C", "Weapon", "Wt.", "Yr.", "season", "sport", "headers", "Bio", "College"]

In [77]:
harvard = harvard[harvardColumns]

In [78]:
harvard.shape

(4270, 19)

In [79]:
yale.shape

(3004, 19)

In [80]:
# Normalize column names
normColumns = ["Name", "B/T", "City", "Events", "High School", "Hometown",
               "Ht.", "No.", "Position", "Region", "S/C", "Weapon", "Wt.", 
               "Class", "Active Seasons", "Sport", "Misc", "Bio", "College"]
yale.columns = normColumns
harvard.columns = normColumns

In [81]:
yale["Hometown"] = yale['Hometown'].apply(lambda x: x.split("/")[0].strip() if type(x )== str else "")

In [82]:
stacked = pd.concat([yale, harvard], axis=0)

In [83]:
# Drop some of the columns we don't need
finalColumns = ["Name", "City", "High School", "Hometown",
                "Ht.", "No.", "Position", "Region", "Wt.",
                "Active Seasons", "Misc", "Bio", "College"]
stacked = stacked[finalColumns]

In [85]:
stacked.head()

,Name,City,High School,Hometown,Ht.,No.,Position,Region,Wt.,Active Seasons,Misc,Bio,College
0,A.J. Haase,Bonne Terre,North County,"Bonne Terre, MO",6-4,NaN,TE,MO,245,2006-10,"{u'Position:': u'G', u'name': u'Josh Greenberg...",2008: Started eight games at TE... LedYale end...,Yale
1,AJ Edwards,Seattle,South Kent School,"Seattle, Wash.",6-5,25,G,Wash.,190,2013-17,"{u'Position:': u'Pole Vault', u'name': u'Brend...",2013-14: Appeared in five games… Scored five p...,Yale
2,Aarica West,Buena Park,Vista Murrieta,"Buena Park, Calif.",5-10,31,G/F,Calif.,NaN,2009-12,"{u'Hometown:': u'Concord, MA', u'High School:'...",Before Yale: Played four years scholasticallyi...,Yale
3,Aaron Ault,Altamonte Springs,Lake Brantley,"Altamonte Springs, FL",6-3,93,LB,FL,194,2012-16,"{u'Position:': u'DB', u'Height:': u'5-11', u'Y...","2013: Played in all 10 games, mostly onspecial...",Yale
4,Aaron Bosgang,Port Washington,Paul D. Schreiber High School,"Port Washington, N.Y.",NaN,NaN,NaN,N.Y.,NaN,2016-17,"{u'Position:': u'OL', u'Height:': u'6-6', u'Ye...",Off the Water: President of National Honor Soc...,Yale


In [22]:
indices = range(0, 7274)
stacked["Student_ID"] = indices

In [23]:
stacked = stacked.set_index(["Student_ID"])

In [24]:
stacked.shape

(7274, 13)

In [86]:
# Split up the active seasons into start and end
stacked["StartSeason"] = stacked.apply(lambda x: int(x["Active Seasons"][0:4]), axis=1)
stacked["EndSeason"] = stacked.apply(lambda x: int(x["ActiveSeasons"][0:2] +
                                                   x["ActiveSeasons"][-2:]),
                                     axis=1)

stacked.drop(["Active Seasons"])

KeyError: ('ActiveSeasons', u'occurred at index 0')

In [25]:
# Seems about right.
processedPath = os.path.join(PROJ_ROOT, "data", "processed", "player_bios_processed.csv")
stacked.to_csv(processedPath, encoding='utf-8')